In [1]:
# How To Perform Inference?

In [2]:
from src.data_preparation.preprocessing.data_util import load_config_file
from src.model.model import SqueezeNet
import torch

C:\Users\Michał\PycharmProjects\SUML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Lod config.yaml file
config_content = load_config_file()

In [4]:
# Prepare model out from config.yaml file
device_str = config_content["train"]["device"] # Name of device, CPU or GPU
device = torch.device(device_str) # CPU or GPU
num_classes = config_content["model"]["num_classes"] # Number of classes
model = SqueezeNet(num_classes=num_classes).to(device) # Model instance

In [6]:
# Load weights
weights = config_content["weights"]
state_dict = torch.load(weights, map_location="cpu", weights_only=True)
model.load_state_dict(state_dict)
model.eval()

<All keys matched successfully>

In [7]:
# Now model is Ready

In [27]:
# How to handle Photo?
from PIL import Image
from src.data_preparation.data_handler.transform import Transforms

path: str = r"C:\Users\Michał\PycharmProjects\SUML\data\3_processed\test\Anthracnose\20211008_124253 (Custom).jpg" # Path to a photo
image = Image.open(path).convert('RGB') # Opening photo
# Preparing Transforms
transform = Transforms()
test_transform = transform.get_test_transform()
# Transforming image
image = test_transform(image)
image = image.unsqueeze(dim=0) # Adding dimensions (3, 224, 224) -> (1, 3, 224, 224)
image = image.to(device) # Putting image on device (GPU or CPU, depends on where model is)
print(type(image))
print(image.size())


<class 'torch.Tensor'>
torch.Size([1, 3, 224, 224])


In [28]:
# Now we have model and prepared image

In [32]:
from src.data_preparation.preprocessing.data_util import load_labels_map
import torch.nn.functional as F

# Load labels map
labels_map = load_labels_map()
print(labels_map)

output = model(image) # inference
probabilities = F.softmax(output, dim=1) # Turn logits to probabilities
# thats it
print(probabilities.detach().cpu().numpy())

{'Anthracnose': 0, 'Bacterial Canker': 1, 'Cutting Weevil': 2, 'Die Back': 3, 'Gall Midge': 4, 'Healthy': 5, 'Powdery Mildew': 6, 'Sooty Mould': 7}
[[9.9970645e-01 2.3969524e-17 2.9357121e-04 1.0207231e-16 9.9423959e-12
  3.5019251e-12 1.0247793e-11 7.3733456e-15]]


In [35]:
# Shwoing probabilities for each class
idx_to_class = {v: k for k, v in labels_map.items()}
probs_flat = probabilities.squeeze()
results = {
    idx_to_class[i]: p * 100
    for i, p in enumerate(probs_flat)
}
for name, pct in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{name:20s}: {pct:.4f}%")


Anthracnose         : 99.9706%
Cutting Weevil      : 0.0294%
Powdery Mildew      : 0.0000%
Gall Midge          : 0.0000%
Healthy             : 0.0000%
Sooty Mould         : 0.0000%
Die Back            : 0.0000%
Bacterial Canker    : 0.0000%
